In [6]:
import finlab
finlab.login(open("config.txt", "r").read())
from finlab import data
import pandas as pd

pd.set_option('display.expand_frame_repr', False)
iStart, stockId = -250, "1503"

closes   = data.get("price:收盤價").fillna(0).astype(float)
opens    = data.get("price:開盤價").fillna(0).astype(float)
highs    = data.get("price:最高價").fillna(0).astype(float)
lows     = data.get("price:最低價").fillna(0).astype(float)
volumns  = data.get("price:成交股數").fillna(0).astype(float)
# kd_k, kd_s  = data.indicator('STOCH', adjust_price=True)

## 三大法人資料 ###
fcb     = data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')
ic      = data.get('institutional_investors_trading_summary:投信買賣超股數')
dealer  = data.get('institutional_investors_trading_summary:自營商買賣超股數(自行買賣)')
dealer2 = data.get('institutional_investors_trading_summary:自營商買賣超股數(避險)')
_v       = data.get('price:成交股數').fillna(0).astype(float)



輸入成功!


INFO:finlab.data:institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商) -- Daily data usage: 727.7 / 5000 MB
INFO:finlab.data:institutional_investors_trading_summary:投信買賣超股數 -- Daily data usage: 733.6 / 5000 MB
INFO:finlab.data:institutional_investors_trading_summary:自營商買賣超股數(自行買賣) -- Daily data usage: 742.8 / 5000 MB
INFO:finlab.data:institutional_investors_trading_summary:自營商買賣超股數(避險) -- Daily data usage: 749.7 / 5000 MB


In [4]:

emaD10 = data.indicator('EMA',adjust_price=True, timeperiod=10, resample="D")  #D 代表日線, W 代表週線, M 代表月線。
emaW13 = data.indicator('EMA',adjust_price=True, timeperiod=13, resample="W")


INFO:finlab.data:etl:adj_close -- Daily data usage: 628.7 / 5000 MB
INFO:finlab.data:etl:adj_open -- Daily data usage: 656.9 / 5000 MB
INFO:finlab.data:etl:adj_high -- Daily data usage: 685.1 / 5000 MB
INFO:finlab.data:etl:adj_low -- Daily data usage: 713.1 / 5000 MB


In [7]:
_emaD10 = emaD10[stockId].iloc[-1]
_emaW13 = emaW13[stockId].iloc[-1]

def format_XQ(r):
    stockId= r["stockId"]
    return stockId+".TW"


from finlab.backtest import sim
data.set_universe(market='TSE_OTC')

emaW01 = closes.resample("W").last()  #計算周K收盤價
emaM01 = closes.resample("M").last()  #計算月K收盤價

# stockId = "1513"
# emaW01[stockId]
cond1 = closes.iloc[-1] > emaW01.iloc[-2]
cond2 = emaW01.iloc[-2] > emaM01.iloc[-2]
cond3 = closes.iloc[-1] < 120
cond4a = fcb.iloc[-1] > 0
cond4b = ic.iloc[-1] > 0
cond = cond1 & cond2 & cond3 & cond4a & cond4b
df = cond[cond]
df.index.tolist()


# ww = emaW01[stockId].iloc[-2]
# mm = emaM01[stockId].iloc[-2]
# cc = closes[stockId].iloc[-1]
# print(stockId, mm,ww,cc)
# rr=sim(cond, resample='W', position_limit=0.05, upload=False)
# print(rr.current_trades)


['1216', '1536', '1736', '2002', '2376', '2377', '2890', '9939']

In [92]:
from finlab.backtest import sim
data.set_universe(market='TSE_OTC')

adj_close   = data.get('etl:adj_close').loc['2012':]
ema_5       = data.indicator('EMA', adjust_price=True, resample='D',timeperiod=5)
ema_10      = data.indicator('EMA', adjust_price=True, resample='D',timeperiod=10)
ema_20      = data.indicator('EMA', adjust_price=True, resample='D',timeperiod=20)
atr         = data.indicator('ATR', adjust_price=True, resample='D',timeperiod=10)
keltner_up  = ema_10+2*atr
vol         = data.get('price:成交股數')/1000
vol_avg     = vol.average(20)

cond1 = (adj_close > keltner_up) & (adj_close.shift() < keltner_up.shift())
cond2 = (ema_5 > ema_10) & (ema_10 > ema_20)
cond3 = vol_avg > 100
entries =  cond1 & cond2 & cond3
exits = adj_close < keltner_up

position = entries.hold_until(exits,nstocks_limit=20, rank=vol_avg)
rr=sim(position,resample='2W',position_limit=0.05,upload=False)
# rr.display()
print(rr.current_trades)

           entry_date exit_date entry_sig_date exit_sig_date  position  period  entry_index  exit_index    return  entry_price  exit_price       mae      gmfe      bmfe       mdd  pdays    weight  next_weights
stock_id                                                                                                                                                                                                         
2634 漢翔    2022-06-20       NaT     2022-06-17    2022-07-03   0.10806     6.0       3738.0        -1.0 -0.074128        34.40         NaN -0.094477  0.000000  0.000000 -0.094477    0.0  0.223264      0.000000
4541 晟田    2022-06-20       NaT     2022-06-17    2022-07-03   0.10806     6.0       3738.0        -1.0  0.020221        27.20         NaN -0.071691  0.047794  0.047794 -0.114035    2.0  0.246015      0.000000
6472 保瑞    2022-06-20       NaT     2022-06-17    2022-07-03   0.10806     6.0       3738.0        -1.0  0.092135       222.50         NaN  0.000000  0.101124  

In [167]:
from finlab import data
import _beowSet as bs  
import _beowFmt as fm 

df = data.get('price:收盤價')
df = df[(df.index >= '2022') & (df.index < "2023")]  # 限定範圍

cond1 = df > df.rolling(5).mean()
cond2 = df > df.rolling(10,min_periods=5).mean()
cond3 = df > df.rolling(20,min_periods=10).mean()
cond4 = df > df.rolling(60,min_periods=40).mean()
cond5 = df > df.rolling(240,min_periods=180).mean()  
position = (df * (cond1 & cond2 & cond3 & cond4))   #多頭排列
position = position[position > 0]                   #找出往上漲
position = position.is_largest(20)                  #前20大的

df = position.iloc[-1]                              #找出最新的一筆(今天收盤後的)
dff = df[df].index.tolist()                         #整理成股票號碼

forXQ = map(lambda x: x+".TW,", dff)                #匯出CSV檔案
stocks, today = "".join(list(forXQ))[:-1], "2022-06-28"
fm.write_LogFile(f"xq_import/{today}_多頭排列.csv", stocks)


In [90]:
from finlab import data,backtest
pb = data.get('price_earning_ratio:股價淨值比')
close = data.get('price:收盤價')
position = (1/(pb * close) * (close > close.average(60)) * (close > 5)).is_largest(20)

rr = backtest.sim(position=-position.astype(float),resample='Q',upload=False)
# rr.benchmark = close['0050']
rr.display()
print(rr.current_trades)

           entry_date exit_date entry_sig_date exit_sig_date  position  period  entry_index  exit_index        return  entry_price  exit_price       mae      gmfe      bmfe       mdd  pdays    weight  next_weights
stock_id                                                                                                                                                                                                             
1217 愛之味   2022-04-01       NaT     2022-03-31    2022-06-30 -0.000881    58.0       3686.0        -1.0 -8.085106e-02        11.75         NaN -0.131915  0.102128  0.102128 -0.212355    7.0  0.041260          0.00
1416 廣豐    2021-10-01       NaT     2021-09-30    2022-06-30 -0.001101   178.0       3566.0        -1.0  4.784689e-02        10.60         NaN  0.000000  0.141509  0.000000 -0.115702  177.0  0.043547          0.00
1453 大將    2021-10-01       NaT     2021-09-30    2022-06-30 -0.001101   178.0       3566.0        -1.0  1.532622e+00         8.43         NaN -